# Disclaimer:

DeepSphere.AI developed these materials based on its team’s expertise and technical infrastructure, and we are sharing these materials strictly for learning and research.
These learning resources may not work on other learning infrastructures and DeepSphere.AI advises the learners to use these materials at their own risk. As needed, we will
be changing these materials without any notification and we have full ownership and accountability to make any change to these materials.

<pre><b>Author :                          Chief Architect :       Reviewer :</b></pre>
<hr>
<pre>Avinash R & Jothi Periasamy       Jothi Periasamy         Jothi Periasamy</pre>

# Import Libraries

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

# Model

In [2]:
class Chatbot:
    def __init__(self, vAR_data):
        self.vAR_data = vAR_data #input json data
        self.vAR_patterns = [] #patterns in input json data
        self.vAR_tags = [] #tags in input json data
        self.vAR_labels = [] #labels in input json data (distinct tags)
        self.vAR_responses = [] #responses in input json data
        self.vAR_num_classes = 0 #total number of classes(tags)
        self.vAR_max_word_size = 1000 #tokenizer keeps the most common 1000 words
        self.vAR_embedding_dim = 16 #output dimension of embedding layer
        self.vAR_max_len = 20 #max length for pad sequence  which is input of embedding layer
        self.vAR_oov_token = "<OOV>" #text to replace when out of vocabulary occurs
        self.vAR_encoder = None #encoder
        self.vAR_tokenizer = None #tokenizer
        self.vAR_padded_sequence = None #padded sequence
        self.vAR_model = None #model
        self.vAR_epochs = 500 #epochs
        
    def extract_data(self):
        #extract tags, patterns, labels from json data
        
        for intent in self.vAR_data['intents']:# loop through input file
            for p in intent['patterns']:#for each elements in patterns
                self.vAR_patterns.append(p)#append each elements in patterns
            
            #append tags n times, where n is number of elements in pattern
            self.vAR_tags.extend([intent['tag']]*len(intent['patterns']))

            self.vAR_responses.append(intent['responses']) #append list of responses
            self.vAR_labels.append(intent['tag']) #append distinct tags
            self.vAR_num_classes = len(self.vAR_labels) #contains number of distinct tags
            
    def encode(self):
        #label encoder
        self.vAR_encoder = LabelEncoder()
        self.vAR_tags = self.vAR_encoder.fit_transform(self.vAR_tags) #encoding each tags
        
    def tokenize(self):
        # Tokenizer- removes punctuation, tokenized
        self.vAR_tokenizer = Tokenizer(num_words=self.vAR_max_word_size, oov_token=self.vAR_oov_token) #tokenizer object
                
        self.vAR_tokenizer.fit_on_texts(self.vAR_patterns) # each word -> unique index (low index - more frequent)
        print('Word Index :', self.vAR_tokenizer.word_index, end='\n')
        
        #convert words in pattern and convert into integer based on self.vAR_tokenizer.word_index
        vAR_sequences = self.vAR_tokenizer.texts_to_sequences(self.vAR_patterns)
        print('Sequence :', vAR_sequences)
        
        #convert integers to 2d array with padding 0.
        #if length exceeds 20 then truncate later elements
        self.vAR_padded_sequences = pad_sequences(vAR_sequences, truncating='post', maxlen=self.vAR_max_len)
        print('Padded Sequence :', self.vAR_padded_sequences)
        
    def create_model(self):
        self.vAR_model = Sequential()
        #Embedding layer - convert each word into a fixed length vector of defined size with real values
        #input_dim : vocabulay size (here 1000)
        #output_dim : each word is represented as a vector of size 16 (here)
        #input_length : length of input padded sequence for this layer is 20
        self.vAR_model.add(Embedding(input_dim=self.vAR_max_word_size, output_dim=self.vAR_embedding_dim, input_length=self.vAR_max_len))
        self.vAR_model.add(GlobalAveragePooling1D())
        self.vAR_model.add(Dense(16, activation='relu'))
        self.vAR_model.add(Dense(16, activation='relu'))
        self.vAR_model.add(Dense(self.vAR_num_classes, activation='softmax'))

        #probability of each class
        self.vAR_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        self.vAR_model.summary()
        
    def train_model(self):
        self.vAR_model.fit(self.vAR_padded_sequences, np.array(self.vAR_tags), epochs=self.vAR_epochs)
        
    def save_objects(self):
        # saving tokenizer
        with open('../Utility/DSAI_Vbot_Tokenizer.pickle', 'wb') as f:
            pickle.dump(self.vAR_tokenizer, f)#, protocol=pickle.HIGHEST_PROTOCOL)

        # saving label encoder
        with open('../Utility/DSAI_Vbot_Label_Encoder.pickle', 'wb') as f:
            pickle.dump(self.vAR_encoder, f)
            
        self.vAR_model.save("../Utility/DSAI_Vbot_ANN_Trained_Model.h5")
        
    def RUN_ALL(self):
        self.extract_data()
        self.encode()
        self.tokenize()
        self.create_model()
        self.train_model()
        self.save_objects()

# Import Data

In [3]:
with open('../Utility/DSAI_Vbot_Intents.json') as file:
    vAR_data = json.load(file)

# Create Object & Run All

In [4]:
chat = Chatbot(vAR_data)
chat.RUN_ALL()

Word Index : {'<OOV>': 1, 'you': 2, 'can': 3, 'i': 4, 'me': 5, 'the': 6, 'how': 7, 'your': 8, 'are': 9, 'what': 10, 'a': 11, 'courses': 12, 'be': 13, 'help': 14, "deepsphere's": 15, 'get': 16, 'for': 17, 'name': 18, 'do': 19, 'send': 20, 'office': 21, 'contact': 22, 'please': 23, 'my': 24, 'done': 25, 'in': 26, "what's": 27, 'see': 28, 'later': 29, 'thanks': 30, 'appreciate': 31, 'it': 32, 'of': 33, 'doing': 34, 'good': 35, 'is': 36, 'have': 37, 'service': 38, 'website': 39, 'number': 40, 'newsletter': 41, 'need': 42, 'support': 43, 'textbooks': 44, 'where': 45, 'order': 46, 'will': 47, 'scheduled': 48, 'only': 49, 'hi': 50, 'hey': 51, 'up': 52, 'there': 53, 'hello': 54, 'hay': 55, 'bye': 56, 'goodbye': 57, 'catch': 58, 'thank': 59, 'so': 60, 'kind': 61, 'really': 62, 'hope': 63, 'whats': 64, 'who': 65, 'call': 66, 'list': 67, "organization's": 68, 'deepsphere': 69, 'link': 70, "link'": 71, 'reach': 72, 'staff': 73, 'signup': 74, 'to': 75, 'guys': 76, 'achievements': 77, 'accomplishmen

2/2 [==============================] - 0s 5ms/step - loss: 2.6320 - accuracy: 0.0893
Epoch 48/500
2/2 [==============================] - 0s 4ms/step - loss: 2.6252 - accuracy: 0.0893
Epoch 49/500
2/2 [==============================] - 0s 4ms/step - loss: 2.6190 - accuracy: 0.0893
Epoch 50/500
2/2 [==============================] - 0s 4ms/step - loss: 2.6131 - accuracy: 0.0893
Epoch 51/500
2/2 [==============================] - 0s 4ms/step - loss: 2.6054 - accuracy: 0.0893
Epoch 52/500
2/2 [==============================] - 0s 2ms/step - loss: 2.5989 - accuracy: 0.0893
Epoch 53/500
2/2 [==============================] - 0s 5ms/step - loss: 2.5917 - accuracy: 0.0893
Epoch 54/500
2/2 [==============================] - 0s 4ms/step - loss: 2.5849 - accuracy: 0.0893
Epoch 55/500
2/2 [==============================] - 0s 4ms/step - loss: 2.5776 - accuracy: 0.0893
Epoch 56/500
2/2 [==============================] - 0s 3ms/step - loss: 2.5693 - accuracy: 0.0893
Epoch 57/500
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 1.7406 - accuracy: 0.5357
Epoch 131/500
2/2 [==============================] - 0s 3ms/step - loss: 1.7299 - accuracy: 0.5536
Epoch 132/500
2/2 [==============================] - 0s 5ms/step - loss: 1.7178 - accuracy: 0.5536
Epoch 133/500
2/2 [==============================] - 0s 3ms/step - loss: 1.7060 - accuracy: 0.5714
Epoch 134/500
2/2 [==============================] - 0s 3ms/step - loss: 1.6947 - accuracy: 0.5893
Epoch 135/500
2/2 [==============================] - 0s 3ms/step - loss: 1.6840 - accuracy: 0.5714
Epoch 136/500
2/2 [==============================] - 0s 3ms/step - loss: 1.6732 - accuracy: 0.5893
Epoch 137/500
2/2 [==============================] - 0s 3ms/step - loss: 1.6619 - accuracy: 0.6071
Epoch 138/500
2/2 [==============================] - 0s 3ms/step - loss: 1.6499 - accuracy: 0.6071
Epoch 139/500
2/2 [==============================] - 0s 3ms/step - loss: 1.6393 - accuracy: 0.6071
Epoch 140/500
2/2 [=====

2/2 [==============================] - 0s 4ms/step - loss: 0.9308 - accuracy: 0.7500
Epoch 213/500
2/2 [==============================] - 0s 3ms/step - loss: 0.9232 - accuracy: 0.7679
Epoch 214/500
2/2 [==============================] - 0s 3ms/step - loss: 0.9161 - accuracy: 0.7679
Epoch 215/500
2/2 [==============================] - 0s 3ms/step - loss: 0.9085 - accuracy: 0.7679
Epoch 216/500
2/2 [==============================] - 0s 3ms/step - loss: 0.9023 - accuracy: 0.7679
Epoch 217/500
2/2 [==============================] - 0s 4ms/step - loss: 0.8942 - accuracy: 0.7679
Epoch 218/500
2/2 [==============================] - 0s 3ms/step - loss: 0.8873 - accuracy: 0.7679
Epoch 219/500
2/2 [==============================] - 0s 4ms/step - loss: 0.8801 - accuracy: 0.7679
Epoch 220/500
2/2 [==============================] - 0s 3ms/step - loss: 0.8737 - accuracy: 0.7679
Epoch 221/500
2/2 [==============================] - 0s 4ms/step - loss: 0.8669 - accuracy: 0.7679
Epoch 222/500
2/2 [=====

2/2 [==============================] - 0s 4ms/step - loss: 0.4886 - accuracy: 0.8750
Epoch 295/500
2/2 [==============================] - 0s 3ms/step - loss: 0.4853 - accuracy: 0.8750
Epoch 296/500
2/2 [==============================] - 0s 4ms/step - loss: 0.4814 - accuracy: 0.8750
Epoch 297/500
2/2 [==============================] - 0s 5ms/step - loss: 0.4774 - accuracy: 0.8750
Epoch 298/500
2/2 [==============================] - 0s 3ms/step - loss: 0.4737 - accuracy: 0.8750
Epoch 299/500
2/2 [==============================] - 0s 4ms/step - loss: 0.4700 - accuracy: 0.8750
Epoch 300/500
2/2 [==============================] - 0s 4ms/step - loss: 0.4663 - accuracy: 0.8750
Epoch 301/500
2/2 [==============================] - 0s 4ms/step - loss: 0.4626 - accuracy: 0.8750
Epoch 302/500
2/2 [==============================] - 0s 3ms/step - loss: 0.4588 - accuracy: 0.8929
Epoch 303/500
2/2 [==============================] - 0s 3ms/step - loss: 0.4553 - accuracy: 0.8929
Epoch 304/500
2/2 [=====

2/2 [==============================] - 0s 4ms/step - loss: 0.2600 - accuracy: 0.9286
Epoch 377/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2579 - accuracy: 0.9286
Epoch 378/500
2/2 [==============================] - 0s 4ms/step - loss: 0.2561 - accuracy: 0.9286
Epoch 379/500
2/2 [==============================] - 0s 5ms/step - loss: 0.2545 - accuracy: 0.9286
Epoch 380/500
2/2 [==============================] - 0s 4ms/step - loss: 0.2526 - accuracy: 0.9286
Epoch 381/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2509 - accuracy: 0.9286
Epoch 382/500
2/2 [==============================] - 0s 4ms/step - loss: 0.2494 - accuracy: 0.9286
Epoch 383/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2478 - accuracy: 0.9286
Epoch 384/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2455 - accuracy: 0.9286
Epoch 385/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2441 - accuracy: 0.9464
Epoch 386/500
2/2 [=====

2/2 [==============================] - 0s 4ms/step - loss: 0.1453 - accuracy: 0.9821
Epoch 459/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1446 - accuracy: 0.9821
Epoch 460/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1434 - accuracy: 0.9821
Epoch 461/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1422 - accuracy: 0.9821
Epoch 462/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1411 - accuracy: 0.9821
Epoch 463/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1403 - accuracy: 0.9821
Epoch 464/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1394 - accuracy: 0.9821
Epoch 465/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1381 - accuracy: 0.9821
Epoch 466/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1374 - accuracy: 0.9821
Epoch 467/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1364 - accuracy: 0.9821
Epoch 468/500
2/2 [=====

# Copyright Notice:

Local and international copyright laws protect
this material. Repurposing or reproducing
this material without written approval from
DeepSphere.AI violates the law.

(c) DeepSphere.AI